In [37]:
#built in 
import os 
import time 
import shutil 
import yaml 
import json
#remote 
import torch

from ultralytics import YOLO


#local 
from dataset.GenerateSyntheticDataset import generate_dataset

In [38]:
#default training checks 
TRAIN = True
GENERATE = True 


#Parameters for  generating  dataset 
NUMBER_OF_IMAGES =10
#segmentation  or detection mode 
SEG =False 
#dataset dir 
SYNTHETIC_DATASET_DIR ='dataset/GeneratedDataset'




# Parameters for training 
EPOCHS  = 2
YAML_DATA_PATH = 'data.yaml'

In [39]:


def add_class_yaml(path_yaml_file:str,new_value,key)->None:
    """
    ADDS CLASS  TO YAML FILE 
    """
    with open(path_yaml_file) as f:
            dic =yaml.load(f,yaml.FullLoader)
           
            if(key in list(dic['names'].keys())):
                print(f"[ERROR] Cant use this key  is already being used {key} ")
            else:
                dic['names'][key]=new_value
            f.close()
            file = open(path_yaml_file,'w')
            yaml.dump(dic,file)
            file.close()
           



In [47]:



def read_depths_display(json_path,yaml_path):
    with open(json_path) as json_file:
        with open(yaml_path) as yaml_file:
            dic_yaml =list(yaml.load(yaml_file,yaml.FullLoader)['names'].values())
            dic_json = json.load(json_file)['NN2_object_depths']['default']
            print(dic_yaml)
            print(dic_json)
            res ={dic_yaml[i]: dic_json [i] for i in range(len(dic_yaml))}
            print(res)



read_depths_display("C:/Users/Testbed/CIIRC_DTU_3WEEK_NEW/cv_pick_place/config/robot_config.json","C:/Users/Testbed/CIIRC_DTU_3WEEK_NEW/cv_pick_place/neural_nets/data.yaml")





['brown', 'small-white', 'medium-white', 'large-white', 'banana', 'catfood', 'ketchup', 'mouthwash', 'showergel', 'skittles', 'stainremover', 'toothpaste', 'trex']
[7.0, 5.0, 5.0, 7.0, 20.0, 10.0, 40.0, 25.0, 45.0, 30.0, 15.0, 35.0]


IndexError: list index out of range

In [23]:
if (GENERATE == True ):
    if(os.path.isdir(SYNTHETIC_DATASET_DIR)):
        shutil.rmtree(SYNTHETIC_DATASET_DIR)
    generate_dataset(NUMBER_OF_IMAGES,SEG)
    

Generating training images
IMAGE NUMBER 0 IS SAVED


KeyboardInterrupt: 

In [ ]:
def train(model_path,seg =True)->None:
    
    
    if(seg==False):
              # use 'yolov8n.pt' to start fresh
            try:
                model = YOLO(model_path)
            except FileNotFoundError:
                print("Model not found, retraining from scratch with YoloV8n")
                model = YOLO('yolov8n.pt',task='detect')
    if(seg == True):
            
            try:
                model = YOLO(model_path)
            except FileNotFoundError:
                print("Model not found, retraining from scratch with YoloV8n")
                model = YOLO('yolov8n-seg.pt',task= 'segment')

     #check if using cuda
    if torch.cuda.is_available():
        print("Using GPU")
        model.cuda()
            
        
    start = time.time()
       
        # if(seg == True ):
        #     results = model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
        # else:
        #      results = model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
    results =model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
       
    end = time.time()
        
       
    print("Training finished")
    val_metrics = model.val()
    val_metrics

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/

In [ ]:
if(TRAIN ==True): 
    #can be any path
    train('runs/detect/train/weights/best.pt',seg=SEG)